In [1]:
import sys
import time
import traceback

import models.mnist_data as mnist
from models.selector import ModelSelector
from modules.hpvconf import HPVGenerator
from modules.hpmgr import HPVManager
from modules.trainmgr import TrainingManager

In [2]:
def run(params, num_convs, num_fcs):
    try:
        print "training starts: " + str(params)
        generator = HPVGenerator()
        generator.setTemplate('CNN_HPV.ini')
        log_path = "../log/cnn_" + "c" + str(num_convs) + "f" + str(num_fcs) + ".csv"
        
        hpv_file = generator.generate(params, output_dir='.config/')
        selector = ModelSelector(mnist.import_dataset())
        model = selector.select(num_convs, num_fcs)
        train_manager = TrainingManager(model, log_path)
        train_manager.setTrainingDevices('gpu', 1)
        train_manager.setLoggingParams([ hyperparam for hyperparam in params])
        hpv = HPVManager(hpv_file, ini_dir='.config/')
        return train_manager.run(hpv)
           
    except:
        e = sys.exc_info()
        print("Error: " + str(e))
        traceback.print_exc()    

In [3]:
total_convs = 96
total_fcs = 512
conv1 = 1
conv2 = 1
conv3 = 1
fc1 = 1
fc2 = 1
fc3 = 1

We will compare the structural differences in following conditions:
 * total number of neurons in convolutional layers: 96
 * total number of neurons in fully connected layers: 512
   * each neurons in fully connected layers will be same due to architectural limitation

In [5]:
convs = [8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88]
num_conv = 2
for conv1 in convs:
    conv2 = total_convs - conv1
    fc = total_fcs
    params = {'CONV1_DEPTH': conv1, 'CONV2_DEPTH': conv2, 'FC1_WIDTH': fc }
    run(params, num_conv, 1)
    fc = total_fcs // 2
    params = {'CONV1_DEPTH': conv1, 'CONV2_DEPTH': conv2, 'FC1_WIDTH': fc, 'FC2_WIDTH': fc }
    run(params, num_conv, 2)
    fc = total_fcs // 3
    params = {'CONV1_DEPTH': conv1, 'CONV2_DEPTH': conv2, 'FC1_WIDTH': fc, 'FC2_WIDTH': fc, 'FC3_WIDTH': fc }
    run(params, num_conv, 3)    

training starts: {'CONV2_DEPTH': 88, 'FC1_WIDTH': 512, 'CONV1_DEPTH': 8}
Training with /root/tf-hpolib/tf-hpo/src/.config/HPV_14794036394189.ini is terminated properly
training starts: {'CONV2_DEPTH': 88, 'FC1_WIDTH': 256, 'CONV1_DEPTH': 8, 'FC2_WIDTH': 256}
Training with /root/tf-hpolib/tf-hpo/src/.config/HPV_14794037247377.ini is terminated properly
training starts: {'CONV2_DEPTH': 88, 'FC3_WIDTH': 170, 'FC1_WIDTH': 170, 'CONV1_DEPTH': 8, 'FC2_WIDTH': 170}
Training with /root/tf-hpolib/tf-hpo/src/.config/HPV_14794038057668.ini is terminated properly
training starts: {'CONV2_DEPTH': 80, 'FC1_WIDTH': 512, 'CONV1_DEPTH': 16}
Training with /root/tf-hpolib/tf-hpo/src/.config/HPV_14794038883433.ini is terminated properly
training starts: {'CONV2_DEPTH': 80, 'FC1_WIDTH': 256, 'CONV1_DEPTH': 16, 'FC2_WIDTH': 256}
Training with /root/tf-hpolib/tf-hpo/src/.config/HPV_14794039832696.ini is terminated properly
training starts: {'CONV2_DEPTH': 80, 'FC3_WIDTH': 170, 'FC1_WIDTH': 170, 'CONV1_DEPTH'

In [8]:
convs = [8, 16, 24, 32, 40, 48, 56, 64, 72, 80]
for conv1 in convs:
    for conv2 in convs:
        conv3 = total_convs - conv1 - conv2
        if conv3 > 0:    
            fc = total_fcs
            params = {'CONV1_DEPTH': conv1, 'CONV2_DEPTH': conv2, 'CONV3_DEPTH': conv3, 'FC1_WIDTH': fc }
            run(params, 3, 1)
            fc = total_fcs // 2
            params = {'CONV1_DEPTH': conv1, 'CONV2_DEPTH': conv2, 'CONV3_DEPTH': conv3, 'FC1_WIDTH': fc, 'FC2_WIDTH': fc }
            run(params, 3, 2)
            fc = total_fcs // 3
            params = {'CONV1_DEPTH': conv1, 'CONV2_DEPTH': conv2, 'CONV3_DEPTH': conv3, 'FC1_WIDTH': fc, 'FC2_WIDTH': fc, 'FC3_WIDTH': fc }
            run(params, 3, 3)

training starts: {'CONV2_DEPTH': 8, 'CONV3_DEPTH': 80, 'FC1_WIDTH': 512, 'CONV1_DEPTH': 8}
Training with /root/tf-hpolib/tf-hpo/src/.config/HPV_14794073462463.ini is terminated properly
training starts: {'CONV2_DEPTH': 8, 'CONV3_DEPTH': 80, 'FC1_WIDTH': 256, 'CONV1_DEPTH': 8, 'FC2_WIDTH': 256}
Training with /root/tf-hpolib/tf-hpo/src/.config/HPV_14794074191461.ini is terminated properly
training starts: {'CONV2_DEPTH': 8, 'FC3_WIDTH': 170, 'CONV1_DEPTH': 8, 'FC1_WIDTH': 170, 'FC2_WIDTH': 170, 'CONV3_DEPTH': 80}
({'conv2_depth': 8, 'dropout_rate': 0.5, 'FC3_WIDTH': 170, 'conv1_depth': 8, 'OPTIMIZATION': 'Momentum', 'MOMENTUM_VALUE': 0.9, 'base_learning_rate': 0.01, 'filter_size': 5, 'init_stddev': 0.1, 'seed': 66478, 'BATCH_SIZE': 64, 'pooling_size': 2, 'num_epochs': 20, 'CONV3_DEPTH': 80, 'eval_frequency': 100, 'eval_batch_size': 64, 'FC1_WIDTH': 170, 'fc1_width': 170, 'fc2_width': 170, 'momentum_value': 0.9, 'STRIDE_SIZE': 2, 'decay_rate': 0.95, 'optimization': 'Momentum', 'POOLING_SI

In [4]:
conv1 = total_convs
fc = total_fcs // 2
params = {'CONV1_DEPTH': conv1, 'FC1_WIDTH': fc, 'FC2_WIDTH': fc }
run(params, 1, 2)
fc = total_fcs // 3
params = {'CONV1_DEPTH': conv1, 'FC1_WIDTH': fc, 'FC2_WIDTH': fc, 'FC3_WIDTH': fc }
run(params, 1, 3)

training starts: {'FC2_WIDTH': 256, 'FC1_WIDTH': 256, 'CONV1_DEPTH': 96}
({'conv2_depth': 64, 'dropout_rate': 0.5, 'FC3_WIDTH': 512, 'conv1_depth': 96, 'OPTIMIZATION': 'Momentum', 'MOMENTUM_VALUE': 0.9, 'base_learning_rate': 0.01, 'filter_size': 5, 'init_stddev': 0.1, 'seed': 66478, 'BATCH_SIZE': 64, 'pooling_size': 2, 'num_epochs': 20, 'CONV3_DEPTH': 128, 'eval_frequency': 100, 'eval_batch_size': 64, 'FC1_WIDTH': 256, 'fc1_width': 256, 'fc2_width': 256, 'momentum_value': 0.9, 'STRIDE_SIZE': 2, 'decay_rate': 0.95, 'optimization': 'Momentum', 'POOLING_SIZE': 2, 'NUM_EPOCHS': 20, 'USE_LEARNING_RATE_DECAY': True, 'EVAL_FREQUENCY': 100, 'EVAL_BATCH_SIZE': 64, 'FC2_WIDTH': 256, 'regularizer_factor': 0.0005, 'batch_size': 64, 'BASE_LEARNING_RATE': 0.01, 'padding': 'SAME', 'FILTER_SIZE': 5, 'INIT_WEIGHT_VALUE': 0.1, 'use_learning_rate_decay': True, 'CONV2_DEPTH': 64, 'DROPOUT_RATE': 0.5, 'fc3_width': 512, 'CONV1_DEPTH': 96, 'DECAY_RATE': 0.95, 'REGULARIZER_FACTOR': 0.0005, 'INIT_STDDEV': 0.1,

99.38